In [1]:

# import core.stock_processor as sp
# import core.plot_processor as pp
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from zigzag import *
from datetime import datetime

import matplotlib.dates as mdates

volume = None
X = None
org_X = None
pivots = None
Ema10 = None
Ema25 = None

Mavol5 = None
Mavol20 = None
Mavol100 = None


Stockname = None
Stockcode = None

def convert_datetime_to_checkpoint(realdata, datetime):
    checkpoint = realdata.loc[realdata['Datetime'] == datetime].index[0]-1
    return checkpoint 

def convert_checkpoint_to_datetime(realdata, checkpoint):
    datetime = realdata.iloc[checkpoint]['Datetime']
    return datetime 

def find_name(stockcode):
    file_name = "code_name.xlsx"
    dfs = pd.read_excel(file_name, sheetname="code_name", )
    dfs.columns =["Code", "Name"]
    dfs.set_index('Code', inplace=True)
    index_stockcode = int(stockcode)
    name = dfs.loc[index_stockcode][0]
    return name

# def create_indicator(df, timeframe, stockcode, datatype):
def create_indicator(df):
    # remove invalid volume data

    # df = df[df['Volume']!=0]
    # parms = talib.ADXR(df['High'], df['Low'], df['Close'], timeperiod=14)
    # parms = talib.APO(df['Close'], fastperiod=12, slowperiod=26, matype=0)
    # bop = talib.BOP(df['Open'],df['High'], df['Low'], df['Close'])

#     ema25 = talib.EMA(df['Close'], timeperiod=25)
#     ema10 = talib.EMA(df['Close'], timeperiod=10)
#     ema5 = talib.EMA(df['Close'], timeperiod=5)

#     mavol5 = talib.EMA(df['Volume'], timeperiod=5)
#     mavol20 = talib.EMA(df['Volume'], timeperiod=20)
#     mavol100 = talib.EMA(df['Volume'], timeperiod=100)

    macd, macdsignal, macdhist = talib.MACD(df['Close'], fastperiod=12, slowperiod=26, signalperiod=9) 
    # rsi = talib.RSI(df['Close'], timeperiod=14)
    # obv = talib.OBV(df['Close'], df['Volume'])
#   \slowk, slowd = talib.STOCH(df['High'], df['Low'], df['Close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)

#     df['Ema25'] = ema25
#     df['Ema10'] = ema10
#     df['Ema5'] = ema5

#     df['Mavol5'] = mavol5
#     df['Mavol20'] = mavol20
#     df['Mavol100'] = mavol100
    
    df['macd'] = macd
    df['macdsignal'] = macdsignal
    df['macdhist'] = macdhist

    # df['Rsi'] = rsi
    # df['Obv'] = obv


#     df['Mavol5'] = df['Mavol5'].fillna(0)
#     df['Mavol20'] = df['Mavol20'].fillna(0)
#     df['Mavol100'] = df['Mavol100'].fillna(0)

#     df['Ema25'] = df['Ema25'].fillna(0)
#     df['Ema10'] = df['Ema10'].fillna(0)
#     df['Ema5'] = df['Ema5'].fillna(0)

    df['macd'] = df['macd'].fillna(0)
    df['macdsignal'] = df['macdsignal'].fillna(0)
    df['macdhist'] = df['macdhist'].fillna(0)
    # df['Rsi'] = df['Rsi'].fillna(0)
    # df['Obv'] = df['Obv'].fillna(0)

    #either fillna or cut first 33 lines.
    df = df[33:]

#     save_to_file(df, timeframe, stockcode, datatype=datatype)
    return df



# def data_prepare(stockcode, datetime_start, datetime_end, timeframe, if_get_new, percentage):
def data_prepare(df, datetime_start, datetime_end, percentage):
    # only need for the first time
    if if_get_new:
        today = datetime.today()
        datem = datetime(today.year, today.month, 1)
        startdate = datem.strftime('%Y-%m-%d')
        
#         if timeframe == "30m":
#             sp.combine_new_data_30m(stockcode=stockcode, startdate=startdate)
#         else:
#             sp.combine_new_data_day(stockcode=stockcode, startdate=startdate)

#     realdata = pd.read_csv('finaldata/current/' + timeframe + '/' + stockcode + '.CSV')
    
    #     realdata_d = pd.read_csv('finaldata/current/' + timeframe + '/' + '.CSV')
    
    #     close price
    global X
    X = np.array(realdata.loc[(realdata['Datetime'] > datetime_start)&(realdata['Datetime'] < datetime_end)]['Close'].tolist())

#     global Ema10
#     Ema10 = np.array(realdata.loc[(realdata['Datetime'] > datetime_start)&(realdata['Datetime'] < datetime_end)]['Ema10'].tolist())

#     global Ema25
#     Ema25 = np.array(realdata.loc[(realdata['Datetime'] > datetime_start)&(realdata['Datetime'] < datetime_end)]['Ema25'].tolist())

#     global Ema5 
#     Ema5 = np.array(realdata.loc[(realdata['Datetime'] > datetime_start)&(realdata['Datetime'] < datetime_end)]['Ema5'].tolist())


#     global Mavol5
#     Mavol5 = np.array(realdata.loc[(realdata['Datetime'] > datetime_start)&(realdata['Datetime'] < datetime_end)]['Mavol5'].tolist())

#     global Mavol20
#     Mavol20 = np.array(realdata.loc[(realdata['Datetime'] > datetime_start)&(realdata['Datetime'] < datetime_end)]['Mavol20'].tolist())

#     global Mavol100 
#     Mavol100 = np.array(realdata.loc[(realdata['Datetime'] > datetime_start)&(realdata['Datetime'] < datetime_end)]['Mavol100'].tolist())



    global org_X
    org_X = np.array(realdata.loc[(realdata['Datetime'] > datetime_start)&(realdata['Datetime'] < datetime_end)])

    global volume
    volume = np.array(realdata.loc[(realdata['Datetime'] > datetime_start)&(realdata['Datetime'] < datetime_end)]['Volume'].tolist())

    global pivots
    pivots = peak_valley_pivots(X, percentage, -percentage)
    
    global Stockname
    Stockname = find_name(stockcode)

    global Stockcode
    Stockcode = stockcode
    
    
def find_breaks():
    breaks = []
    combine_list=np.column_stack((np.arange(len(X))[pivots!=0], X[pivots != 0]))
    for i in np.arange(len(combine_list)-4):
        current_max = np.amax(combine_list[[i,i+1],[1]])
        next_min = np.amin(combine_list[[i+2,i+3],[1]])
        current_min = np.amin(combine_list[[i,i+1],[1]])
        next_max = np.amax(combine_list[[i+2,i+3],[1]])

        if next_min > current_max :
#             this is up
#             composite index, datetime, next_index, next_date
            breaks.append( [combine_list[i+1].tolist(), org_X[combine_list[i+1][0].astype(int)][0], combine_list[i+2].tolist(), org_X[combine_list[i+2][0].astype(int)][0], "up"] )
        elif current_min > next_max :
#             this is down
            breaks.append( [combine_list[i+1].tolist(), org_X[combine_list[i+1][0].astype(int)][0], combine_list[i+2].tolist(), org_X[combine_list[i+2][0].astype(int)][0], "Down"] )
    return breaks        


def find_xy(element, breaks, i):
    if i == 0:
        box_start = 0
        x = 0
    else:
        box_start = int(breaks[i-1][2][0])   
        x = int( breaks[i-1][2][0] )
    
    box_end = int(breaks[i][0][0])
    y = np.amin(X[box_start : box_end])


    return (x, y)


def find_height(element, breaks, i):
    if i == 0:
        box_start = 0
    else:
        box_start = int(breaks[i-1][2][0])   

    box_end = int(breaks[i][0][0])
    low = np.amin(X[box_start : box_end])
    high = y = np.amax(X[box_start : box_end])
    height = high - low

    return height, high, low


def find_width(element, breaks, i):
    if i == 0:
        box_start = 0
    else:
        box_start = int(breaks[i-1][2][0])   

    box_end = int(breaks[i][0][0])
    width = abs(box_end - box_start)

    return width
    
    
def plot_pivots():
    plt.figure(figsize=(16,8))
    plt.xlim(0, len(X))
    plt.ylim(X.min()*0.99, X.max()*1.01)
    plt.plot(np.arange(len(X)), X, 'k:', alpha=0.8)
    plt.plot(np.arange(len(X))[pivots != 0], X[pivots != 0], 'k-', linewidth=.1 )
    plt.scatter(np.arange(len(X))[pivots == 1], X[pivots == 1], color='r', s=1)
    plt.scatter(np.arange(len(X))[pivots == -1], X[pivots == -1], color='g', s=1)
    
    breaks = find_breaks()
    
    for i,element in enumerate(breaks):
        xy = find_xy(element, breaks, i)
        height = find_height(element, breaks, i)
        width = find_width(element, breaks, i)
        plt.gca().add_patch( plt.Rectangle(xy, width, height, fill=False, edgecolor='g', linewidth=0.8) )
    
    
def new_plot():
    fig = plt.figure(facecolor='white',figsize=(16,8))
    fig.suptitle( u'' + Stockname + " " + Stockcode,fontsize=30, fontproperties="SimHei") # SimSun

    left, width = 0.0, 0.8
    rect1 = [left, 0.3, width, 0.7] # left, bottom, width, height
    ax1 = fig.add_axes(rect1)
    ax1.set_xlim(0, len(X)+10)
    ax1.set_ylim(X.min()*0.99, X.max()*1.01)
    ax1.plot(np.arange(len(X)), X, 'k:', alpha=1)
    ax1.plot(np.arange(len(X))[pivots != 0], X[pivots != 0], 'k-', linewidth=.5 )
    ax1.scatter(np.arange(len(X))[pivots == 1], X[pivots == 1], color='r', s=10)
    ax1.scatter(np.arange(len(X))[pivots == -1], X[pivots == -1], color='g', s=10)

#     ax1.plot(np.arange(len(X)), Ema5, 'r-', alpha=0.5, linewidth=.5)
#     ax1.plot(np.arange(len(X)), Ema10, 'g-', alpha=0.5, linewidth=.5)
#     ax1.plot(np.arange(len(X)), Ema25, 'b-', alpha=0.5, linewidth=.5)



    breaks = find_breaks()
    last_xy, last_height, last_width, last_low, last_high = deal_with_current(breaks)

    #draw box.
    for i,element in enumerate(breaks):
        xy = find_xy(element, breaks, i)
        height_r, high, low = find_height(element, breaks, i)
        width_r = find_width(element, breaks, i)
        #         plt.gca().add_patch( plt.Rectangle(xy, width, height, fill=False, edgecolor='g', linewidth=0.8) )
        ax1.add_patch( plt.Rectangle(xy, width_r, height_r, fill=False, edgecolor='g', linewidth=0.8) )

        ax1.annotate(high, (xy[0]+1, high+0.01), color='r')
        ax1.annotate(low, (xy[0]+1, low-0.06), color='g')

    ax1.add_patch( plt.Rectangle(last_xy, last_width, last_height, fill=False, edgecolor='r', linewidth=0.8) )    
    ax1.annotate(last_high, (last_xy[0]+1, last_high+0.01), color='r')
    ax1.annotate(last_low, (last_xy[0]+1, last_low-0.06), color='g')



    fillcolor = 'darkgoldenrod'
    rect2 = [left, 0.1, width, 0.2]
    ax2 = fig.add_axes(rect2, sharex=ax1)
    #     ax2.fill_between(np.arange(len(X)), volume, 0, facecolor=fillcolor, edgecolor=fillcolor)
    ax2.bar(np.arange(len(X)), volume, color=fillcolor, width=0.8, align='center')

    ax2.plot(np.arange(len(X)), Mavol5, 'r-', alpha=1, linewidth=.9)
    ax2.plot(np.arange(len(X)), Mavol20, 'g-', alpha=1, linewidth=.9)
    ax2.plot(np.arange(len(X)), Mavol100, 'b-', alpha=1, linewidth=.9)

    plt.show()

def deal_with_current(breaks):

    last_pivots = int(breaks[-1][2][0])
    x = np.arange(len(X))[pivots != 0]
    last_box = x[x>last_pivots][:-1]
    last_xy = (last_box[0], np.amin(X[last_box]))
    last_height = np.amax(X[last_box]) - np.amin(X[last_box])
    last_width = last_box[-1] - last_box[0]
    last_low = np.amin(X[last_box])
    last_high = np.amax(X[last_box])
    
    return last_xy, last_height, last_width, last_low, last_high







In [2]:
realdata = pd.read_csv('data/current/' + timeframe + '/' + stockcode + '.CSV')

new_plot()

NameError: name 'timeframe' is not defined